# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [80]:
import numpy as np

In [81]:
import pandas as pd

In [82]:
import math

In [83]:
import requests

In [117]:
import xlsxwriter

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [85]:
stocks=pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [86]:
from secrets1 import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [87]:
symbol = 'A'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
print(api_url)
data=requests.get(api_url).json()
type(data)
#print(data.status_code)
print(data)

https://sandbox.iexapis.com/stable/stock/A/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448
{'avgTotalVolume': 1746687, 'calculationPrice': 'close', 'change': 0.62, 'changePercent': 0.0052, 'close': 0, 'closeSource': 'oiiflfca', 'closeTime': None, 'companyName': 'Agilent Technologies Inc.', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': 'y li1penu dactmd5e erei', 'highTime': 1660084932265, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 126.01, 'iexCloseTime': 1660458426885, 'iexLastUpdated': 1689787228862, 'iexMarketPercent': 0.0450472539799995, 'iexOpen': 128, 'iexOpenTime': 1730279157157, 'iexRealtimePrice': 124.84, 'iexRealtimeSize': 104, 'iexVolume': 58824, 'lastTradeTime': 1707616043567, 'latestPrice': 127.71, 'latestSource': 'Close', 'latestTime': 'July 11, 2022', 'latestUpdate': 1694688845423, 'l

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [88]:
price = data['latestPrice']
print(price)
market_cap=data['marketCap']
print(market_cap/1000000000000)

127.71
0.03785265243


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [89]:
my_columns=['Ticker','Stock Price','Market Capitalization', 'Number of Shares to Buy']
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [90]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index=my_columns
    ),
    ignore_index=True
)    

/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_38465/235980403.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.71,37852652430,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [91]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol in stocks['Ticker'][:2]:
    api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data['latestPrice'],
                data['marketCap'],
                'N/A',
            ],
            index=my_columns,
        ),
        ignore_index=True,
    )

/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_38465/850250756.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_38465/850250756.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


In [92]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.66,37845467011,N/A
1,AAL,13.70,8859770463,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [93]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]
        #print('DISCA, HFC, VIAC, WLTW')

In [97]:
symbol_groups=list(chunks(stocks['Ticker'], 100))
#symbol_groups
symbol_strings=[] # create empty list of strings
for i in range(0, len(symbol_groups)):  # indexes through all the 100-item lists in the symbol_groups
    #print(i) # gives number of lists
    #print(symbol_groups[i])
    symbol_strings.append(','.join(symbol_groups[i]) ) # use join mtd to appendsitems in RH lists  to LH of strings
    #print(symbol_strings[i]) # this is a list symbol_strings of 100-stock strings

final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings: # [5:6]
    #print(symbol_string)  
    #api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    #print(batch_api_call_url)
    #data=requests.get(batch_api_call_url)
    #print(data.status_code) # check if successful
    data=requests.get(batch_api_call_url).json() # put output in json obj
    for symbol in symbol_string.split(','): # split symbol_string csv-like obj and index thru
        #print(symbol)
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'NA'
            ],
            index=my_columns),
            ignore_index=True,
        )
final_dataframe

/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_38465/2496904560.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_38465/2496904560.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_38465/2496904560.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/k8/qrg9qn2n1mg3q14f5qwqx2600000gn/T/ipykernel_38465/2496904560.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fi

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.09,38295666194,NA
1,AAL,13.56,8816001329,NA
2,AAP,187.05,11342543695,NA
3,AAPL,148.24,2450118448487,NA
4,ABBV,156.25,284211595061,NA
...,...,...,...,...
496,YUM,116.66,33593855388,NA
497,ZBH,106.50,22122113431,NA
498,ZBRA,292.20,15804696990,NA
499,ZION,50.50,7885111833,NA


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [101]:
while True:
    portfolio_size = input('Enter the value of your portfolio. Or type "n" to exit. ')
    if portfolio_size == 'n':
        break
    else:
        try:
            val = float(portfolio_size)
            break
        except ValueError:
            print('Please enter a number.')

print('You entered a portfolio value of: ', portfolio_size, 'dollar(s).')

You entered a portfolio value of:  10000000 dollar(s).


In [114]:
numStocks = len(final_dataframe.index)
#print(numStocks)

position_size = val/len(final_dataframe.index)
position_size
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'] )

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.09,38295666194,155
1,AAL,13.56,8816001329,1471
2,AAP,187.05,11342543695,106
3,AAPL,148.24,2450118448487,134
4,ABBV,156.25,284211595061,127
...,...,...,...,...
496,YUM,116.66,33593855388,171
497,ZBH,106.50,22122113431,187
498,ZBRA,292.20,15804696990,68
499,ZION,50.50,7885111833,395


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [158]:

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer=pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
final_dataframe.to_excel(writer, 'Recommended Trades', index=False) # 

# Close the Pandas Excel writer and output the Excel file.
#writer.save()


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [159]:
background_color = '#0a0a23'
font_color='#ffffff'

string_format=writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
    }
)

dollar_format=writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'num_format': '$0.00',
    }
)

integer_format=writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'num_format': '0',
    }
)

cap_format=writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'num_format': '$00,000',
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [152]:
# writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
# writer.sheets['Recommended Trades'].set_column('B:B',18,dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C',18,cap_format)
# writer.sheets['Recommended Trades'].set_column('D:D',18,cap_format)
# writer.save()
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', cap_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', cap_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [160]:
column_formats={
'A': ['Ticker', string_format],
'B': ['Stock Price', dollar_format],
'C': ['Market Capitalization', cap_format],
'D': ['Number of Shares to Buy', integer_format], 
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save() 